In [2]:
import numpy as np
import pandas as pd
import nbformat
import re
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import openai
from openpyxl import Workbook
from openpyxl.styles import Font
import os
from dotenv import load_dotenv
from pprint import pprint
from openai import OpenAI
from fpdf import FPDF

In [3]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key is None:
    raise ValueError("API key is not set")

# Inicializar la API de OpenAI
openai.api_key = openai_api_key

In [4]:
def listar_notebooks(directory):
    """
    Devuelve una lista con los nombres de todos los archivos con extensión '.ipynb' en un directorio dado,
    excluyendo aquellos que se llamen 'solucion.ipynb'.
    
    Parameters:
        directory (str): La ruta al directorio donde buscar los archivos.
        
    Returns:
        list: Una lista con los nombres de los archivos que cumplen los criterios.
    """
    archivos = []
    for filename in os.listdir(directory):
        if filename.endswith('.ipynb') and filename != 'solucion.ipynb':
            archivos.append(filename)
            
    alumnos = [archivo.replace('.ipynb', '') for archivo in archivos]
    return alumnos, archivos

In [5]:
def extrae_criterios(prompt_file):
    """
    Extrae los nombres de los criterios de un archivo de plantilla de prompt.

    Parámetros:
    prompt_file (str): Ruta al archivo que contiene la plantilla de prompt con los criterios delimitados por '@@'.

    Retorna:
    list: Una lista de cadenas, donde cada cadena es el nombre de un criterio extraído de la plantilla de prompt.
    """
    # Abrir y leer el archivo de plantilla de prompt
    with open(prompt_file, 'r', encoding='utf-8') as file:
        prompt_template = file.read()

    # Buscar los nombres de los criterios usando una expresión regular
    criteria_pattern = re.compile(r'@@(.*?)@@')
    criteria = criteria_pattern.findall(prompt_template)
    
    return criteria

In [6]:
def preprocesa_notebook(file_path):
    """
    Procesa un notebook Jupyter para extraer el contexto del examen, los enunciados de los ejercicios, el código de solución para cada ejercicio y el nombre del alumno.
    
    Parameters:
        file_path (str): Ruta del archivo del notebook de solución.
        
    Returns:
        dict: Un diccionario con las claves 'contexto_examen', 'enunciados_ejercicios', 'codigo_ejercicios', y 'alumno'.
    """
    # Inicialización de variables
    contexto_examen = ""
    enunciados_ejercicios = []
    codigo_ejercicios = []

    # Leer el notebook
    with open(file_path, 'r', encoding='utf-8') as f:
        notebook_tmp = nbformat.read(f, as_version=4)
        
        # Procesar las celdas del notebook
        for cell in notebook_tmp.cells:
            if cell.cell_type == 'markdown':
                cell_content = cell['source'].strip()
                
                if cell_content.startswith('Contexto'):
                    # Asignar el contenido de la celda a la variable contexto_examen
                    contexto_examen = cell_content
                
                elif cell_content.startswith('Ejercicio'):
                    # Añadir el contenido de la celda a la lista enunciados_ejercicios
                    enunciados_ejercicios.append(cell_content)
            
            elif cell.cell_type == 'code':
                # Añadir el código de cada celda de código a codigo_ejercicios
                codigo_ejercicios.append(cell['source'])

    # Separar código en ejercicios utilizando el marcador '# Solucion ejercicio'
    codigo_ejercicios = re.split(r'# Solucion ejercicio \d+', '\n'.join(codigo_ejercicios))
    codigo_ejercicios = [code.strip() for code in codigo_ejercicios if code.strip()]  # Limpiar y filtrar vacíos

    # Extraer el nombre del alumno del nombre del archivo
    nombre_archivo = os.path.basename(file_path)  # Obtener el nombre del archivo con extensión
    alumno, _ = os.path.splitext(nombre_archivo)  # Separar el nombre y la extensión
    
    return {
        'contexto_examen': contexto_examen,
        'enunciados_ejercicios': enunciados_ejercicios,
        'codigo_ejercicios': codigo_ejercicios,
        'alumno': alumno
    }

In [7]:
def evaluar_ejecucion_ejercicios(datos_ejercicios):
    """
    Evalúa la ejecución de los ejercicios en el diccionario de datos extraídos del notebook.
    
    Parameters:
        datos_ejercicios (dict): Diccionario con 'contexto_examen', 'enunciados_ejercicios' y 'codigo_ejercicios'.
        
    Returns:
        dict: Un diccionario con los datos originales y el estado de ejecución y los mensajes de error para cada ejercicio.
    """
    resultados = []
    codigo_ejercicios = datos_ejercicios['codigo_ejercicios']
    
    for idx, code in enumerate(codigo_ejercicios):
        resultado = {
            'id_ejercicio': idx + 1,
            'ejecucion': False,
            'mensaje_error': '',
            'calificacion': 0  # Calificación inicial, puede ser actualizada después
        }
        try:
            exec(code)
            resultado['ejecucion'] = True
            resultado['calificacion'] = 1  # Calificación inicial de 1 para ejecuciones exitosas
        except Exception as e:
            resultado['mensaje_error'] = str(e)
        
        resultados.append(resultado)
    
    # Incluir los resultados en la estructura de datos original
    datos_ejercicios_con_resultados = {
        'contexto_examen': datos_ejercicios['contexto_examen'],
        'enunciados_ejercicios': datos_ejercicios['enunciados_ejercicios'],
        'codigo_ejercicios': datos_ejercicios['codigo_ejercicios'],
        'resultados': resultados
    }
    
    return datos_ejercicios_con_resultados

# Ejemplo de uso
# datos_ejercicios = preprocesa_notebook("ruta/a/tu/archivo.ipynb")
# datos_evaluados = evaluar_ejecucion_ejercicios(datos_ejercicios)
# print(datos_evaluados)


In [8]:
def evaluar_con_chatgpt(codigo, descripcion, prompt_template):
    """
    Evalúa el código de un ejercicio utilizando el modelo GPT-4 de OpenAI.

    Parameters:
        codigo (str): El código del ejercicio a evaluar.
        descripcion (str): Una descripción del ejercicio.
        prompt_template (str): El template del prompt con marcadores para la descripción y el código.

    Returns:
        dict: Un diccionario con la puntuación y comentario del ejercicio.
    """
    # Insertar los valores en el prompt
    prompt = prompt_template.format(descripcion=descripcion, codigo=codigo)

    cliente = OpenAI()
    response = cliente.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a programming teaching assistant evaluating student code."},
            {"role": "user", "content": prompt}
        ]
    )

    # Extraer la respuesta de ChatGPT
    evaluacion = response.choices[0].message.content
    return evaluacion

In [9]:
def extract_criteria(prompt):
    """
    Extrae los nombres de los criterios de un prompt dado.

    Parámetros:
    prompt (str): Cadena de texto que contiene el prompt con los criterios delimitados por '@@'.

    Retorna:
    list: Una lista de cadenas, donde cada cadena es el nombre de un criterio extraído del prompt.
    """
    # Buscar los nombres de los criterios
    criteria_pattern = re.compile(r'@@(.*?)@@')
    criteria = criteria_pattern.findall(prompt)
    return criteria

In [10]:
def evaluar_ejercicios(diccionario_resultados, prompt_file='prompt.txt'):
    """
    Evalúa los ejercicios utilizando GPT-4 y devuelve las notas y comentarios para cada ejercicio.
    
    Parameters:
        diccionario_resultados (dict): Diccionario con los datos del notebook preprocesado.
        prompt_file (str): Ruta del archivo de texto que contiene el prompt.

    Returns:
        dict: Un diccionario con las evaluaciones de cada ejercicio.
    """
    # Leer el prompt desde el archivo y cerrar el archivo automáticamente
    with open(prompt_file, 'r', encoding='utf-8') as file:
        prompt_template = file.read()
        
    evaluaciones = {}
    for i, (enunciado, codigo) in enumerate(zip(diccionario_resultados['enunciados_ejercicios'], diccionario_resultados['codigo_ejercicios']), start=1):
        # Llamar a la función que evalúa con ChatGPT pasando el prompt template
        resultado = evaluar_con_chatgpt(codigo, enunciado, prompt_template)
        evaluaciones[f'Ejercicio {i}'] = resultado
    
    return evaluaciones

In [11]:
import re

def extraer_resultados(resultado):
    """
    Extrae las puntuaciones, comentarios y comentarios generales de un diccionario de resultados.

    Parámetros:
    resultado (dict): Diccionario que contiene los resultados en forma de texto.

    Retorna:
    dict: Un diccionario con las claves originales y sus correspondientes listas de puntuaciones, comentarios y comentarios generales.
    """
    # Inicializar el diccionario para almacenar los resultados
    res = {}
    
    # Recorrer cada elemento del diccionario 'resultado'
    for key, value in resultado.items():
        # Buscar el patrón para las puntuaciones usando una expresión regular
        puntuaciones_pattern = re.search(r'A\.\s\*\*Puntuaciones\*\*:\s(\[.*?\])', value)
        # Buscar el patrón para los comentarios usando una expresión regular
        comentarios_pattern = re.search(r'B\.\s\*\*Comentarios\*\*:\s(.*?)C\.\s\*\*Comentario General\*\*:', value, re.DOTALL)
        # Buscar el patrón para el comentario general usando una expresión regular
        comentario_general_pattern = re.search(r'C\.\s\*\*Comentario General\*\*:\s(.*)', value, re.DOTALL)
        
        # Si se encuentra el patrón de puntuaciones, evalúa la cadena como una lista
        if puntuaciones_pattern:
            puntuaciones = eval(puntuaciones_pattern.group(1))
        else:
            puntuaciones = []

        # Si se encuentra el patrón de comentarios, evalúa la cadena como una lista
        if comentarios_pattern:
            comentarios_text = comentarios_pattern.group(1).strip()
            # Encuentra todos los comentarios dentro del texto de comentarios
            comentarios = re.findall(r'"\s*(.*?)\s*"', comentarios_text, re.DOTALL)
        else:
            comentarios = []

        # Si se encuentra el patrón de comentario general, evalúa la cadena como una lista
        if comentario_general_pattern:
            comentario_general = eval(comentario_general_pattern.group(1).strip())
        else:
            comentario_general = []

        # Añadir el resultado al diccionario 'res' para la clave actual
        res[key] = {
            'puntuaciones': puntuaciones,
            'comentarios': comentarios,
            'comentario_general': comentario_general
        }

    return res


In [12]:
# Clase personalizada para el PDF
class PDF(FPDF):
    """
    Clase personalizada para crear reportes de evaluación de ejercicios en formato PDF.
    
    Atributos:
    alumno (str): Nombre del alumno para el cual se está generando el reporte.

    Métodos:
    __init__(self, alumno): Inicializa la instancia de la clase PDF con el nombre del alumno.
    header(self): Añade un encabezado a cada página del PDF con el nombre del alumno.
    footer(self): Añade un pie de página a cada página del PDF con el número de página.
    add_context(self, contexto_examen): Añade el contexto del examen al PDF.
    add_enunciados(self, enunciados): Añade los enunciados de los ejercicios al PDF.
    add_evaluacion(self, alumno, ejercicios, criterios): Añade la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
    """
    
    def __init__(self, alumno):
        """
        Inicializa la instancia de la clase PDF con el nombre del alumno.
        
        Parámetros:
        alumno (str): Nombre del alumno para el cual se está generando el reporte.
        """
        super().__init__()
        self.alumno = alumno
    
    def header(self):
        """
        Añade un encabezado a cada página del PDF con el nombre del alumno.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, f'Reporte de Evaluación de Ejercicios de {self.alumno}', 0, 1, 'C')

    def footer(self):
        """
        Añade un pie de página a cada página del PDF con el número de página.
        """
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    def add_context(self, contexto_examen):
        """
        Añade el contexto del examen al PDF.
        
        Parámetros:
        contexto_examen (str): Contexto general del examen.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Contexto del Examen:', 0, 1)
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, contexto_examen)
        self.ln(10)

    def add_enunciados(self, enunciados):
        """
        Añade los enunciados de los ejercicios al PDF.
        
        Parámetros:
        enunciados (list): Lista de enunciados de los ejercicios del examen.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Enunciados de los Ejercicios:', 0, 1)
        self.set_font('Arial', '', 12)
        for i, enunciado in enumerate(enunciados, 1):
            self.cell(0, 10, f'Ejercicio {i}:', 0, 1)
            self.multi_cell(0, 10, enunciado)
            self.ln(5)
        self.ln(10)

    def add_evaluacion(self, alumno, ejercicios, criterios):
        """
        Añade la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
        
        Parámetros:
        alumno (str): Nombre del alumno.
        ejercicios (dict): Diccionario que contiene las evaluaciones de los ejercicios del estudiante.
        criterios (list): Lista de criterios de evaluación.
        """
        self.set_font('Arial', 'B', 16)
        self.cell(0, 10, f'Informe de {alumno}', 0, 1, 'C')
        self.ln(10)
   
        for ejercicio, contenido in ejercicios.items():
            # Título del ejercicio
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, ejercicio, 0, 1)
            self.ln(5)
            
            # Puntuaciones y comentarios
            for i, criterio in enumerate(criterios):
                self.set_font('Arial', 'B', 12)
                self.cell(0, 10, f'{criterio}:', 0, 1, 'L')
                self.set_font('Arial', '', 12)
                self.multi_cell(0, 10, f'Puntuación: {contenido["puntuaciones"][i]}')
                self.multi_cell(0, 10, f'Comentario: {contenido["comentarios"][i]}')
                self.ln(5)
            
            # Comentario general
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Comentario General del Ejercicio:', 0, 1, 'L')
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, contenido['comentario_general'][0])
            self.ln(10)


In [13]:
# Función para crear el PDF
def create_pdf(file_path, student_name, contexto_examen, enunciados, ejercicios, criterios):
    """
    Crea un archivo PDF con la evaluación del estudiante.

    Parámetros:
    file_path (str): Ruta donde se guardará el archivo PDF.
    student_name (str): Nombre del estudiante.
    contexto_examen (str): Contexto general del examen.
    enunciados (list): Lista de enunciados de los ejercicios del examen.
    ejercicios (dict): Diccionario que contiene las evaluaciones de los ejercicios del estudiante. La estructura del diccionario es:
        {
            'Ejercicio 1': {
                'puntuaciones': [listado de puntuaciones],
                'comentarios': [listado de comentarios],
                'comentario_general': [comentario general]
            },
            'Ejercicio 2': {...},
            ...
        }
    criterios (list): Lista de criterios de evaluación.

    Procedimiento:
    1. Crea una instancia de la clase PDF con el nombre del estudiante.
    2. Añade una página al PDF.
    3. Agrega el contexto del examen al PDF.
    4. Agrega los enunciados de los ejercicios al PDF.
    5. Agrega la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
    6. Guarda el archivo PDF en la ruta especificada.

    Retorna:
    Ninguno
    """
    
    pdf = PDF(student_name)
    pdf.add_page()
    
    # Agregar contexto del examen
    pdf.add_context(contexto_examen)
    
    # Agregar enunciados de los ejercicios (comentado porque no está en uso)
    # pdf.add_enunciados(enunciados)
    
    # Agregar evaluación del estudiante
    pdf.add_evaluacion(student_name, ejercicios, criterios)
    
    # Guardar el PDF con el nombre del estudiante
    pdf.output(file_path)

In [14]:
def generar_pdfs_para_estudiantes(directorio_examen, archivo_examen, directorio_reports, resultados):
    """
    Genera archivos PDF de evaluación para cada estudiante basado en los resultados del examen.

    Parámetros:
    directorio_examen (str): Ruta al directorio donde se encuentra el archivo del examen.
    archivo_examen (str): Nombre del archivo del examen.
    directorio_reports (str): Ruta al directorio donde se guardarán los archivos PDF generados.
    resultados (dict): Diccionario que contiene las evaluaciones de los estudiantes. La estructura del diccionario es:
        {
            'nombre_estudiante': {
                'Ejercicio 1': {
                    'puntuaciones': [listado de puntuaciones],
                    'comentarios': [listado de comentarios],
                    'comentario_general': [comentario general]
                },
                'Ejercicio 2': {...},
                ...
            },
            ...
            'criterios': [listado de criterios]
        }

    Precondiciones:
    - El archivo del examen debe existir en la ruta especificada.
    - Las funciones `preprocesa_notebook` y `create_pdf` deben estar definidas previamente.

    Procedimiento:
    1. Preprocesa el archivo del examen para extraer el contexto y los enunciados de los ejercicios.
    2. Crea el directorio para guardar los archivos PDF si no existe.
    3. Para cada estudiante en los resultados, genera un archivo PDF con las evaluaciones y comentarios.
    """
    
    # Preprocesar el archivo de examen
    file_path = os.path.join(directorio_examen, archivo_examen)
    examen_preprocesado = preprocesa_notebook(file_path)
    
    # Crear directorio para guardar los PDFs
    if not os.path.exists(directorio_reports):
        os.makedirs(directorio_reports)
    
    # Generar el PDF para cada estudiante
    for student_name, ejercicios in resultados.items():
        if student_name != 'criterios':  # Ignorar la clave de criterios
            file_path = os.path.join(directorio_reports, f'{student_name}.pdf')
            create_pdf(file_path, student_name, examen_preprocesado['contexto_examen'], examen_preprocesado['enunciados_ejercicios'], ejercicios, resultados['criterios'])


In [17]:
def generar_excel_resultados(resultados, filename='resultados_evaluacion.xlsx'):
    filas = []

    for alumno, ejercicios in resultados.items():
        if alumno == 'criterios':  # Ignorar la clave de criterios
            continue
        fila = {'Alumno': alumno}
        notas = []
        for ejercicio, contenido in ejercicios.items():
            media = np.mean(contenido['puntuaciones'])
            fila[ejercicio] = media
            notas.append(media)
        
        # Calcular la nota final como la media ponderada de las notas de los ejercicios
        nota_final = np.mean(notas)
        fila['NOTA FINAL'] = nota_final
        
        filas.append(fila)

    df = pd.DataFrame(filas)
    df.to_excel(filename, index=False)
    print(f"Archivo Excel '{filename}' generado con éxito.")

_______________________________

In [15]:
def main(directorio_raiz):
    # Construir las rutas basadas en el directorio raíz
    directorio_entregas = os.path.join(directorio_raiz, "entregas")
    prompt_file = os.path.join(directorio_raiz, 'prompt.txt')
    directorio_examen = os.path.join(directorio_raiz, "examenes")
    directorio_reports = os.path.join(directorio_raiz, "reports")
    fichero_res = 'resultados_evaluacion.xlsx'

    # Listar los notebooks entregados por los alumnos
    alumnos, ficheros = listar_notebooks(directorio_entregas)

    # Extraer los criterios de evaluación del prompt
    criterios = extrae_criterios(prompt_file)

    # Inicializar el diccionario de resultados
    resultados = {}
    resultados['criterios'] = criterios

    # Procesar y evaluar cada notebook
    for indice, fich in enumerate(ficheros):
        nb_preprocesado = preprocesa_notebook(os.path.join(directorio_entregas, fich))
        res_eval_tmp = evaluar_ejercicios(nb_preprocesado, prompt_file=prompt_file)
        res_extraido = extraer_resultados(res_eval_tmp)
        resultados[alumnos[indice]] = res_extraido
        
    # Generar los informes en PDF para cada estudiante
    generar_pdfs_para_estudiantes(directorio_examen, 'examen.ipynb', directorio_reports, resultados)

    # Generar el archivo Excel con los resultados de la evaluación
    generar_excel_resultados(resultados, filename=os.path.join(directorio_reports, fichero_res))


In [1]:
!ls "/workspace/entregas"

evaluacion.xlsx       ubeda_fernando.ipynb
santos_alfonso.ipynb  ventura_pedro.ipynb


In [17]:
directorio_entregas = os.path.join(directorio_raiz, "entregas")
directorio_entregas

'/workspace/entregas'

In [18]:
directorio_raiz = "/workspace"
main(directorio_raiz)

Archivo Excel '/workspace/reports/resultados_evaluacion.xlsx' generado con éxito.


In [14]:
directorio = "/workspace/tests"

prompt_file = '/workspace/prompt.txt'

directorio_examen = "/workspace/examenes"

directorio_reports = "/workspace/reports"

fichero_res = 'resultados_evaluacion.xlsx'

alumnos, ficheros = listar_notebooks(directorio)

criterios = extrae_criterios(prompt_file)

resultados = {}

resultados['criterios'] = criterios

for indice, fich in enumerate(ficheros):
    nb_preprocesado = preprocesa_notebook(os.path.join(directorio, fich))
    res_eval_tmp = evaluar_ejercicios(nb_preprocesado, prompt_file='workspace/prompt.txt')
    res_extraido = extraer_resultados(res_eval_tmp)
    resultados[alumnos[indice]] = res_extraido
    
    
generar_pdfs_para_estudiantes(directorio_examen, 'examen.ipynb', directorio_reports, resultados)

generar_excel_resultados(resultados, filename='workspace/reports/resultados_evaluacion.xlsx')

In [15]:
directorio_examen = "/workspace/examenes"

directorio_reports = "/workspace/reports"

generar_pdfs_para_estudiantes(directorio_examen, 'examen.ipynb', directorio_reports, resultados)

generar_excel_resultados(resultados, filename='workspace/reports/resultados_evaluacion.xlsx')

________________

In [21]:
def generar_excel_resultados(resultados, filename='resultados_evaluacion.xlsx'):
    filas = []

    for alumno, ejercicios in resultados.items():
        if alumno == 'criterios':  # Ignorar la clave de criterios
            continue
        fila = {'Alumno': alumno}
        notas = []
        for ejercicio, contenido in ejercicios.items():
            media = np.mean(contenido['puntuaciones'])
            fila[ejercicio] = media
            notas.append(media)
        
        # Calcular la nota final como la media ponderada de las notas de los ejercicios
        nota_final = np.mean(notas)
        fila['NOTA FINAL'] = nota_final
        
        filas.append(fila)

    df = pd.DataFrame(filas)
    df.to_excel(filename, index=False)
    print(f"Archivo Excel '{filename}' generado con éxito.")

In [22]:
generar_excel_resultados(resultados, filename='workspace/reports/resultados_evaluacion.xlsx')

Archivo Excel 'workspace/reports/resultados_evaluacion.xlsx' generado con éxito.


In [18]:
resultados

{'criterios': ['Exactitud',
  'Claridad',
  'Eficiencia',
  'Creatividad/Alternativas',
  'Documentación/Comentarios',
  'Robustez/Resiliencia',
  'Cumplimiento de las Instrucciones',
  'Estructura del Código'],
 'santos_alfonso': {'Ejercicio 1': {'puntuaciones': [10,
    8,
    10,
    5,
    5,
    5,
    10,
    5],
   'comentarios': ['El código produce la salida correcta y no presenta errores. La lista de saldos y el saldo total acumulado se calculan correctamente.',
    'El código es bastante legible y las variables tienen nombres significativos. Sin embargo, podría mejorarse con más comentarios explicando el paso del cálculo de saldos.',
    'El código es eficiente y utiliza un bucle simple para calcular los saldos, lo que es apropiado para esta cantidad de datos.',
    'La solución es básica y directa. Se podría mejorar usando comprensión de listas para hacer el código más conciso.',
    'El código carece de comentarios descriptivos sobre el propósito y la lógica de cada sección

__________________________________________________________________

# MANTENIDO POR SI GENERAN ERRORES LOS DE ARRIBA

In [11]:
# Clase personalizada para el PDF
class PDF(FPDF):
    """
    Clase personalizada para crear reportes de evaluación de ejercicios en formato PDF.
    
    Atributos:
    alumno (str): Nombre del alumno para el cual se está generando el reporte.

    Métodos:
    __init__(self, alumno): Inicializa la instancia de la clase PDF con el nombre del alumno.
    header(self): Añade un encabezado a cada página del PDF con el nombre del alumno.
    footer(self): Añade un pie de página a cada página del PDF con el número de página.
    add_context(self, contexto_examen): Añade el contexto del examen al PDF.
    add_enunciados(self, enunciados): Añade los enunciados de los ejercicios al PDF.
    add_evaluacion(self, alumno, ejercicios, criterios): Añade la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
    """
    
    def __init__(self, alumno):
        """
        Inicializa la instancia de la clase PDF con el nombre del alumno.
        
        Parámetros:
        alumno (str): Nombre del alumno para el cual se está generando el reporte.
        """
        super().__init__()
        self.alumno = alumno
    
    def header(self):
        """
        Añade un encabezado a cada página del PDF con el nombre del alumno.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, f'Reporte de Evaluación de Ejercicios de {self.alumno}', 0, 1, 'C')

    def footer(self):
        """
        Añade un pie de página a cada página del PDF con el número de página.
        """
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    def add_context(self, contexto_examen):
        """
        Añade el contexto del examen al PDF.
        
        Parámetros:
        contexto_examen (str): Contexto general del examen.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Contexto del Examen:', 0, 1)
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, contexto_examen)
        self.ln(10)

    def add_enunciados(self, enunciados):
        """
        Añade los enunciados de los ejercicios al PDF.
        
        Parámetros:
        enunciados (list): Lista de enunciados de los ejercicios del examen.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Enunciados de los Ejercicios:', 0, 1)
        self.set_font('Arial', '', 12)
        for i, enunciado in enumerate(enunciados, 1):
            self.cell(0, 10, f'Ejercicio {i}:', 0, 1)
            self.multi_cell(0, 10, enunciado)
            self.ln(5)
        self.ln(10)

    def add_evaluacion(self, alumno, ejercicios, criterios):
        """
        Añade la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
        
        Parámetros:
        alumno (str): Nombre del alumno.
        ejercicios (dict): Diccionario que contiene las evaluaciones de los ejercicios del estudiante.
        criterios (list): Lista de criterios de evaluación.
        """
        self.set_font('Arial', 'B', 16)
        self.cell(0, 10, f'Informe de {alumno}', 0, 1, 'C')
        self.ln(10)
        
        for ejercicio, contenido in ejercicios.items():
            # Título del ejercicio
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, ejercicio, 0, 1)
            self.ln(5)
            
            # Puntuaciones y comentarios
            for i, criterio in enumerate(criterios):
                self.set_font('Arial', 'B', 12)
                self.cell(0, 10, f'{criterio}:', 0, 1, 'L')
                self.set_font('Arial', '', 12)
                self.multi_cell(0, 10, f'Puntuación: {contenido["puntuaciones"][i]}')
                self.multi_cell(0, 10, f'Comentario: {contenido["comentarios"][i]}')
                self.ln(5)
            
            # Comentario general
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Comentario General del Ejercicio:', 0, 1, 'L')
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, contenido['comentario_general'][0])
            self.ln(10)


In [16]:
# Función para crear el PDF
def create_pdf(file_path, student_name, contexto_examen, enunciados, ejercicios, criterios):
    """
    Crea un archivo PDF con la evaluación del estudiante.

    Parámetros:
    file_path (str): Ruta donde se guardará el archivo PDF.
    student_name (str): Nombre del estudiante.
    contexto_examen (str): Contexto general del examen.
    enunciados (list): Lista de enunciados de los ejercicios del examen.
    ejercicios (dict): Diccionario que contiene las evaluaciones de los ejercicios del estudiante. La estructura del diccionario es:
        {
            'Ejercicio 1': {
                'puntuaciones': [listado de puntuaciones],
                'comentarios': [listado de comentarios],
                'comentario_general': [comentario general]
            },
            'Ejercicio 2': {...},
            ...
        }
    criterios (list): Lista de criterios de evaluación.

    Procedimiento:
    1. Crea una instancia de la clase PDF con el nombre del estudiante.
    2. Añade una página al PDF.
    3. Agrega el contexto del examen al PDF.
    4. Agrega los enunciados de los ejercicios al PDF.
    5. Agrega la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
    6. Guarda el archivo PDF en la ruta especificada.

    Retorna:
    Ninguno
    """
    
    pdf = PDF(student_name)
    pdf.add_page()
    
    # Agregar contexto del examen
    pdf.add_context(contexto_examen)
    
    # Agregar enunciados de los ejercicios (comentado porque no está en uso)
    # pdf.add_enunciados(enunciados)
    
    # Agregar evaluación del estudiante
    pdf.add_evaluacion(student_name, ejercicios, criterios)
    
    # Guardar el PDF con el nombre del estudiante
    pdf.output(file_path)

In [19]:
def generar_pdfs_para_estudiantes(directorio_examen, archivo_examen, directorio_reports, resultados):
    """
    Genera archivos PDF de evaluación para cada estudiante basado en los resultados del examen.

    Parámetros:
    directorio_examen (str): Ruta al directorio donde se encuentra el archivo del examen.
    archivo_examen (str): Nombre del archivo del examen.
    directorio_reports (str): Ruta al directorio donde se guardarán los archivos PDF generados.
    resultados (dict): Diccionario que contiene las evaluaciones de los estudiantes. La estructura del diccionario es:
        {
            'nombre_estudiante': {
                'Ejercicio 1': {
                    'puntuaciones': [listado de puntuaciones],
                    'comentarios': [listado de comentarios],
                    'comentario_general': [comentario general]
                },
                'Ejercicio 2': {...},
                ...
            },
            ...
            'criterios': [listado de criterios]
        }

    Precondiciones:
    - El archivo del examen debe existir en la ruta especificada.
    - Las funciones `preprocesa_notebook` y `create_pdf` deben estar definidas previamente.

    Procedimiento:
    1. Preprocesa el archivo del examen para extraer el contexto y los enunciados de los ejercicios.
    2. Crea el directorio para guardar los archivos PDF si no existe.
    3. Para cada estudiante en los resultados, genera un archivo PDF con las evaluaciones y comentarios.
    """
    
    # Preprocesar el archivo de examen
    file_path = os.path.join(directorio_examen, archivo_examen)
    examen_preprocesado = preprocesa_notebook(file_path)
    
    # Crear directorio para guardar los PDFs
    if not os.path.exists(directorio_reports):
        os.makedirs(directorio_reports)
    
    # Generar el PDF para cada estudiante
    for student_name, ejercicios in resultados.items():
        if student_name != 'criterios':  # Ignorar la clave de criterios
            file_path = os.path.join(directorio_reports, f'{student_name}.pdf')
            create_pdf(file_path, student_name, examen_preprocesado['contexto_examen'], examen_preprocesado['enunciados_ejercicios'], ejercicios, resultados['criterios'])


In [20]:

directorio_examen = "/workspace/examenes"

generar_pdfs_para_estudiantes(directorio_examen, 'examen.ipynb', '/workspace/reports', resultados) #, create_pdf, preprocesa_notebook)

# file_path = os.path.join(directorio_examen, 'examen.ipynb')

# examen_preprocesado = preprocesa_notebook(file_path)

In [75]:
# Crear directorio para guardar los PDFs
directorio_reports = "/workspace/reports"
if not os.path.exists(directorio_reports):
    os.makedirs(directorio_reports)

# Generar el PDF para cada estudiante
for student_name, ejercicios in resultados.items():
    if student_name != 'criterios':  # Ignorar la clave de criterios
        file_path = os.path.join(directorio_reports, f'{student_name}.pdf')
        create_pdf(file_path, student_name, examen_preprocesado['contexto_examen'], examen_preprocesado['enunciados_ejercicios'], ejercicios, resultados['criterios'])


In [92]:
def generar_excel_resultados(resultados, filename='resultados_evaluacion.xlsx'):
    # Crear una lista para almacenar los resultados en formato tabular
    filas = []

    # Recorrer los resultados para cada estudiante
    for alumno, ejercicios in resultados.items():
        fila = {'Alumno': alumno}
        for ejercicio, contenido in ejercicios.items():
            media = np.mean(contenido['puntuaciones'])
            fila[ejercicio] = media
        filas.append(fila)

    # Crear un DataFrame de pandas con los datos
    df = pd.DataFrame(filas)

    # Guardar el DataFrame en un archivo Excel
    df.to_excel(filename, index=False)

    print(f"Archivo Excel '{filename}' generado con éxito.")
    
    generar_excel_resultados(resultados, filename='workspace/reports/resultados_evaluacion.xlsx')


In [89]:
resultados.items()

dict_items([('criterios', ['Exactitud', 'Claridad', 'Eficiencia', 'Creatividad/Alternativas', 'Documentación/Comentarios', 'Robustez/Resiliencia', 'Cumplimiento de las Instrucciones', 'Estructura del Código']), ('santos_alfonso', {'Ejercicio 1': {'puntuaciones': [5, 8, 9, 6, 4, 5, 10, 5], 'comentarios': ['El código produce la salida correcta y no genera errores.', 'El código es legible y está bien estructurado, con nombres de variables claros.', 'El código es eficiente, utilizando un único bucle para calcular los saldos.', 'La solución es directa, aunque se podría considerar el uso de comprensión de listas para simplificar aún más el código.', 'Faltan comentarios que expliquen la lógica del cálculo de saldos.', 'El código no presenta excepciones, ya que las listas son del mismo tamaño y están correctamente definidas.', 'El código sigue todas las instrucciones del ejercicio sin dejar ningún requerimiento sin atender.', 'La estructura del código es básica y funcional, aunque no se utiliz

In [88]:
list(resultados.keys())[1]

'santos_alfonso'

In [80]:
generar_excel_resultados(resultados, filename='workspace/reports/resultados_evaluacion.xlsx')

AttributeError: 'list' object has no attribute 'items'